In [ ]:
import os
import json
import numpy as np
import matplotlib.pyplot as plt
import scipy as sp
from scipy.io import wavfile
from scipy.signal import fftconvolve
import IPython
import pyroomacoustics as pra
from pyroomacoustics.denoise import apply_spectral_sub, apply_iterative_wiener
from scipy import signal as sps


TRAIN_DIR = "/home/karim/Desktop/Sonos_Assignment/vad_train_set/"
TEST_DIR = "/home/karim/Desktop/Sonos_Assignment/vad_test_set/"

## Apply Spatial Processing
In this section, I apply different spatial processing techniques on the input signals. I tested with the following

*Beamforming*: 1) delay-and-sum (das) 2) minimum variance distortionless response (mvdr)

*Denoising*: 1) spectral subtraction 2) iterative wiener filtering

*Bandpass Filtering*: 30Hz-3KHz voice band

In [ ]:
# Defining a room through rt60 
def get_room(rt60_tgt, fs): 
    # Setting up room dims
    room_dim = [3, 3, 3]  # meters (Assuming a cubic room of 3x3x3 meters - some rt60 were too low for bigger)
    room_center = [dim / 2 for dim in room_dim]
    # put our michrophone in the center of the room
    R = np.array([[room_center[0] - (0.071/2), room_center[0] + (0.071/2)], 
                  [room_center[1], room_center[1]], 
                  [room_center[2], room_center[2]]])  # [[x], [y], [z]]
    # Setting up room based on rt60
    e_absorption, max_order = pra.inverse_sabine(rt60_tgt, room_dim)
    room_bf = pra.ShoeBox(room_dim, fs=fs, materials=pra.Material(e_absorption), max_order=max_order)
    return room_bf, R, room_center

In [ ]:
# Main Processing Loop 
def process_data_dir(top_directory, beamformer = "mvdr"):
    directory = top_directory + "audio/"
    for filename in os.listdir(directory):
        if filename.endswith(".wav"):
            # Load files
            audio_file = os.path.join(directory, filename)
            meta_file = os.path.join(top_directory + "metadata/", filename[:-3]+"json")
            fs, signal = wavfile.read(audio_file)
            signal = pra.normalize(signal.astype(float))
            with open(meta_file) as json_file:
                meta_data = json.load(json_file)
                
            # Setting up room based on rt60
            rt60_tgt = float(meta_data['rt60'])
            room_bf, R, room_center = get_room(rt60_tgt, fs)
            
            # Load azimuth and elevation
            source_azimuth, source_elevation =  meta_data['source_doa']
            noise_azimuth, noise_elevation = meta_data['noise_doa']
                
            # Putting sources in room
            # Assume all sources are 1 meter away from the center (but if SNR is pure speech to noise, 
            # maybe we can set the distance for each source proportional to the SNR)
            r = 1 
            # Setup sources for speech and noise 
            speech_source = [r*np.cos(source_azimuth)*np.cos(source_elevation) + room_center[0],
                            r*np.sin(source_azimuth)*np.cos(source_elevation) + room_center[1],
                            r*np.sin(source_elevation) + room_center[2]] # [x,y,z]
            noise_source = [r*np.cos(noise_azimuth)*np.cos(noise_elevation) + room_center[0],
                            r*np.sin(noise_azimuth)*np.cos(noise_elevation) + room_center[1],
                            r*np.sin(noise_elevation) + room_center[2]] # [x,y,z]
            
            # Compute noise variance [Not sure about this bit], is SNR from the source speech?? 
            # Maybe better use the first 3 seconds for that? 
            SNR = meta_data['SNR']
            if (SNR != None):
                SNR = int(SNR)
                mono_signal = 0.5 * (signal[:,0] + signal[:,1])
                # compute Signal power [not correct if SNR is pure speech to noise instead of noisy speech to noise]
                P_s = sp.sum(mono_signal*mono_signal)/mono_signal.size 
                sigma2_n = P_s / 10**(SNR/10) # compute noise power
            else:
                sigma2_n = 5e-7 # set small value for noise power if SNR is None

            # Here I pass a dummy signal to the source just to simulate. We ignore this signal later
            room_bf.add_source(speech_source, delay=0., signal=signal.T[0,:])
            room_bf.add_source(noise_source, delay=0, signal=np.zeros_like(signal.T[0,:]))
            
            # define our beamformer
            Lg_t = 0.100                # filter size in seconds
            Lg = np.ceil(Lg_t*fs)       # in samples
            fft_len = 512
            mics = pra.Beamformer(R, room_bf.fs, N=fft_len, Lg=Lg)
            room_bf.add_microphone_array(mics)
            
            # Choose beamformer algorithm
            if (beamformer == "das"):
                mics.rake_delay_and_sum_weights(room_bf.sources[0][:1],room_bf.sources[1][:1])
            elif (beamformer == "mvdr"):
                mics.rake_mvdr_filters(room_bf.sources[0][:1] , room_bf.sources[1][:1] , sigma2_n * 
                                       np.eye(mics.Lg * mics.M))
            elif (beamformer == "percuptual"):
                mics.rake_perceptual_filters(room_bf.sources[0][0:1] , room_bf.sources[1][0:1] , 
                                             sigma2_n * np.eye(mics.Lg * mics.M))

            # Run simulation
            room_bf.compute_rir()
            room_bf.simulate()

            # Replace microphone signals with the recorded signals in our dataset, i.e. ignoring dummy sources
            room_bf.mic_array.record(signal.T, fs)

            # Get enhanced signal
            beamform_signal = mics.process(FD=False)
            beamform_signal = pra.normalize(beamform_signal)
            
            #Apply denoising 
            # 1) Spectral Subtractio
            denoised_beamform_signal_spectral = apply_spectral_sub(beamform_signal, nfft=512,
                                         db_reduc=12, lookback=15, beta=20, alpha=3)
            denoised_beamform_signal_spectral = pra.normalize(denoised_beamform_signal_spectral)
            # 2) iterative wiener
            denoised_beamform_signal_wiener = apply_iterative_wiener(beamform_signal, frame_len=512,
                                             lpc_order=20, iterations=2,
                                             alpha=0.8, thresh=0.01)
            denoised_beamform_signal_wiener = pra.normalize(denoised_beamform_signal_wiener)
            
            # Apply narrowband speech filtering (does not really seem to make things better, but can save memory)
            sos = sps.butter(10, [30, 3000], 'bandpass', fs=fs, output='sos')
            # Spectral signal
            filtered_denoised_beamform_signal_spectral = sps.sosfilt(sos, denoised_beamform_signal_spectral)
            filtered_denoised_beamform_signal_spectral = pra.normalize(filtered_denoised_beamform_signal_spectral)
            # Wiener signal
            filtered_denoised_beamform_signal_wiener = sps.sosfilt(sos, denoised_beamform_signal_wiener)
            filtered_denoised_beamform_signal_wiener = pra.normalize(filtered_denoised_beamform_signal_wiener)

            # Downsampling to 6kHz [This is very slow, maybe I can do it with librosa when reading the file later]
            # number_of_samples = round(filtered_denoised_beamform_signal.shape[0] * float(6000) / fs)
            # downsampled_filtered_denoised_beamform_signal = sps.resample(filtered_denoised_beamform_signal, 
            #                                                             number_of_samples)

            # Saving all version of the signal
            # Only beamformed 
            wavfile.write(top_directory + beamformer + "/[beamformed]" + filename, 
              fs, beamform_signal)
            
            # Beamformed + Denoise (spectral and wiener)
            wavfile.write(top_directory + beamformer + "_spectral/[spectral_beamformed]" + filename, 
              fs, denoised_beamform_signal_spectral)      
            wavfile.write(top_directory + beamformer + "_wiener/[wiener_beamformed]" + filename, 
              fs, filtered_denoised_beamform_signal_wiener)
            
            # Beamformed + Denoise (spectral and wiener) + Speech-Band Filtered
            wavfile.write(top_directory + beamformer + "_spectral_filtered/[filtered_spectral_beamformed]" + filename, 
                          fs, filtered_denoised_beamform_signal_spectral)
            wavfile.write(top_directory + beamformer + "_wiener_filtered/[filtered_wiener_beamformed]" + filename, 
                          fs, filtered_denoised_beamform_signal_wiener)

In [ ]:
# Process both the train and test data
beamforming_methods = ["das", "mvdr"]
#beamforming_methods = ["das", "mvdr", "percuptual"]
for beamformer in beamforming_methods:
    process_data_dir(TRAIN_DIR, beamformer)
    process_data_dir(TEST_DIR, beamformer)

## Listen to some samples

In [ ]:
# Load all the  processed  files for a sample audio file
TRAIN_DIR = "/home/karim/Desktop/Sonos_Assignment/vad_train_set/"
sample_file = "f1_0a4e47f1-184a-473a-a466-64154ff4703f.wav" #Here is where to choose which file
original = TRAIN_DIR + "audio/" + sample_file
das = TRAIN_DIR + "das/" + "[beamformed]" + sample_file
das_spectral = TRAIN_DIR + "das_spectral/" + "[spectral_beamformed]" + sample_file
das_spectral_filtered = TRAIN_DIR + "das_spectral_filtered/" + "[filtered_spectral_beamformed]" + sample_file
das_wiener = TRAIN_DIR + "das_wiener/" + "[wiener_beamformed]" + sample_file
das_wiener_filtered = TRAIN_DIR + "das_wiener_filtered/" + "[filtered_wiener_beamformed]" + sample_file
mdvr = TRAIN_DIR + "mvdr/" + "[beamformed]" + sample_file
mdv_spectral = TRAIN_DIR + "mvdr_spectral/" + "[spectral_beamformed]" + sample_file
mdv_spectral_filtered = TRAIN_DIR + "mvdr_spectral_filtered/" + "[filtered_spectral_beamformed]" + sample_file
mdvr_wiener = TRAIN_DIR + "mvdr_wiener/" + "[wiener_beamformed]" + sample_file
mdvr_wiener_filtered = TRAIN_DIR + "mvdr_wiener_filtered/" + "[filtered_wiener_beamformed]" + sample_file

fs, signal_original = wavfile.read(original)
fs, signal_das = wavfile.read(das)
fs, signal_das_spectral = wavfile.read(das_spectral)
fs, signal_das_spectral_filtered = wavfile.read(das_spectral_filtered)
fs, signal_das_wiener = wavfile.read(das_wiener)
fs, signal_das_wiener_filtered = wavfile.read(das_wiener_filtered)
fs, signal_mdvr = wavfile.read(mdvr)
fs, signal_mdv_spectral = wavfile.read(mdv_spectral)
fs, signal_mdv_spectral_filtered = wavfile.read(mdv_spectral_filtered)
fs, signal_mdvr_wiener = wavfile.read(mdvr_wiener)
fs, signal_mdvr_wiener_filtered = wavfile.read(mdvr_wiener_filtered)

In [ ]:
print("The original audio")
IPython.display.Audio(0.5*(signal_original[:,0] + signal_original[:,1]), rate=fs)

In [ ]:
print("delay-and-sum output")
IPython.display.Audio(signal_das, rate=fs)

In [ ]:
print("delay-and-sum + spectral-subtraction output")
IPython.display.Audio(signal_das_spectral, rate=fs)

In [ ]:
print("delay-and-sum + spectral-subtraction + bandbass filter output")
IPython.display.Audio(signal_das_spectral_filtered, rate=fs)

In [ ]:
print("delay-and-sum + iterative wiener output")
IPython.display.Audio(signal_das_wiener, rate=fs)

In [ ]:
print("delay-and-sum + iterative wiener + bandbass filter output")
IPython.display.Audio(signal_das_wiener_filtered, rate=fs)

In [ ]:
print("mvdr output")
IPython.display.Audio(signal_mdvr, rate=fs)

In [ ]:
print("mvdr + spectral-subtraction output")
IPython.display.Audio(signal_mdv_spectral, rate=fs)

In [ ]:
print("mvdr + spectral-subtraction + bandbass filter output")
IPython.display.Audio(signal_mdv_spectral_filtered, rate=fs)

In [ ]:
print("mvdr + iterative wiener output")
IPython.display.Audio(signal_mdvr_wiener, rate=fs)

In [ ]:
print("mvdr + iterative wiener + bandbass filter output")
IPython.display.Audio(signal_mdvr_wiener_filtered, rate=fs)